In [2]:
import torch
import torch.nn.functional as F
from models import Create_nets
from datasets import get_dataloader
#from options import TrainOptions
from torchvision import models
import os
from PIL import Image
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np

import cv2
from utils.results import *
from matplotlib.cm import viridis
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from utils.dataloader import get_paths_mvtec
from datasets import ImageDataset_mvtec
from torch.utils.data import DataLoader
import copy

In [3]:
import os
import json

class TrainOptions:
    def __init__(self):
        self.exp_name = "DEV_USDR"
        self.epoch_start = 0
        self.epoch_num = 1
        self.factor = 1
        self.seed = 233
        self.fixed_seed_bool = True
        self.test_seed = 400
        self.num_row = 4
        self.activation = 'gelu'
        self.unalign_test = False
        self.data_root = '/home/bule/projects/datasets/mvtec_anomaly_detection/'
        self.data_category = "cable"
        self.data_set = "mvtec"
        self.batch_size = 2
        self.lr = 1e-4
        self.b1 = 0.5
        self.b2 = 0.999
        self.n_cpu = 8
        self.results_dir = 'results'
        self.image_result_dir = 'result_images'
        self.model_result_dir = 'saved_models'
        self.validation_image_dir = 'validation_images'
        self.contamination_rate = 0.1
        self.assumed_contamination_rate = 0.1
        self.mode = 'mvtec'
        self.development = False
        self.parser = None
        self.initialized = False


In [4]:
args = TrainOptions()
os.makedirs(args.results_dir, exist_ok=True)
os.makedirs(os.path.join(args.results_dir, args.data_set, f'contamination_{int(args.contamination_rate*100)}', f'{args.exp_name}-{args.data_category}', args.image_result_dir), exist_ok=True)
os.makedirs(os.path.join(args.results_dir, args.data_set, f'contamination_{int(args.contamination_rate*100)}', f'{args.exp_name}-{args.data_category}', args.model_result_dir), exist_ok=True)

try:
    with open(os.path.join('configurations', f'{args.data_set}.json'), 'r') as file:
        dataset_parameters = json.load(file)
    setattr(args, 'dataset_parameters', dataset_parameters)
except FileNotFoundError:
    print(f"Configuration file for {args.data_set} not found. Proceeding with default parameters.")
    setattr(args, 'dataset_parameters', {})
    
torch.manual_seed(args.seed)

In [5]:
normal_images, validation_images, sampled_anomalies_for_train, sampled_anomalies_for_val, good_images_test, remaining_anomalies_test = get_paths_mvtec(args,verbose=True)
DATA_PATH=os.path.join(args.data_root,args.data_category)
# combine good and anomalies
train_data=normal_images+sampled_anomalies_for_train

train_dataloader = DataLoader(ImageDataset_mvtec(args,DATA_PATH,mode='train',train_paths = train_data,test_paths = None),
                                                batch_size=args.batch_size,shuffle=True,num_workers=args.n_cpu,drop_last=False)

category: cable, normals train:  224, anomalies test: 92, normal test: 58
anomalies test total:     {'bent_wire': 13, 'cable_swap': 12, 'combined': 11, 'cut_inner_insulation': 14, 'cut_outer_insulation': 10, 'missing_cable': 12, 'missing_wire': 10, 'poke_insulation': 10}
anomalies test sampled:   {'bent_wire': 3, 'cable_swap': 2, 'combined': 2, 'cut_inner_insulation': 3, 'cut_outer_insulation': 2, 'missing_cable': 2, 'missing_wire': 2, 'poke_insulation': 2}
anomalies test remaining: {'bent_wire': 10, 'cable_swap': 10, 'combined': 9, 'cut_inner_insulation': 11, 'cut_outer_insulation': 8, 'missing_cable': 10, 'missing_wire': 8, 'poke_insulation': 8}


In [19]:
np.random.seed(args.seed)
N_samples = len(train_data)
print(N_samples)
idx = np.arange(N_samples)
np.random.shuffle(idx)
idx


stride =45
window_size = 120
# Create train sets
shifts = np.floor(N_samples / stride)
train_sets = int(shifts)
print('train_sets: ', train_sets)
print('window_size: ', window_size)
train_ind_ls = []

for i in range(train_sets):
    train_ind_ls.append(idx[np.arange(i * stride, i * stride + window_size) % N_samples])



recon_error_ls = []

## Preset model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

backbone = models.resnet18(pretrained=True).to(device)
backbone.eval()
outputs = []
def hook(module, input, output):
    outputs.append(output)
backbone.layer1[-1].register_forward_hook(hook)
backbone.layer2[-1].register_forward_hook(hook)
backbone.layer3[-1].register_forward_hook(hook)

criterion = nn.MSELoss()

def embedding_concat(x, y):
    B, C1, H1, W1 = x.size()
    _, C2, H2, W2 = y.size()
    s = int(H1 / H2)
    x = F.unfold(x, kernel_size=s, dilation=1, stride=s)
    x = x.view(B, C1, -1, H2, W2)
    z = torch.zeros(B, C1 + C2, x.size(2), H2, W2).to(device)
    for i in range(x.size(2)):
        z[:, :, i, :, :] = torch.cat((x[:, :, i, :, :], y), 1)
    z = z.view(B, -1, H2 * W2)
    z = F.fold(z, kernel_size=s, output_size=(H1, W1), stride=s)

    return z



for j in range(train_sets):
    
    model_result_m=args.model_result_dir+'_j_'+str(j)
    
    EXPERIMENT_PATH = os.path.join(args.results_dir,args.data_set ,f'contamination_{int(args.contamination_rate*100)}',f'{args.exp_name}-{args.data_category}')
    SAVE_DIR= os.path.join(EXPERIMENT_PATH, model_result_m, 'checkpoint.pth')
    
    if not os.path.exists(os.path.join(EXPERIMENT_PATH, model_result_m)):
        os.makedirs(os.path.join(EXPERIMENT_PATH, model_result_m))

    paths_j = [train_data[idx] for idx in train_ind_ls[j]]

    dataset_j=ImageDataset_mvtec(args,DATA_PATH,mode='train',train_paths = paths_j, test_paths = None)
    train_j_dataloader = DataLoader(dataset_j, batch_size=2,shuffle=True,num_workers=8,drop_last=False)
    
    #train_dataloader, valid_loader ,test_dataloader = get_dataloader(args)

    ## Train model
    
    # #model....
    start_epoch = 0
    transformer = Create_nets(args)
    transformer = transformer.to(device)
    transformer.cuda()
    optimizer = torch.optim.Adam( transformer.parameters(), lr=args.lr, betas=(args.b1, args.b2))
    best_loss = 1e10
    
    
    #### TRAIN MODEL
    for epoch in range(start_epoch, args.epoch_num):
        avg_loss = 0
        avg_loss_scale = 0
        total = 0
        transformer.train()
        for i,(filename, batch) in enumerate(train_j_dataloader):
            
            inputs = batch.to(device)
            outputs = []
            optimizer.zero_grad()



            with torch.no_grad():
                _ = backbone(inputs)

                outputs = embedding_concat(embedding_concat(outputs[0],outputs[1]),outputs[2])
        
            recon, std = transformer(outputs)
            torch.cuda.empty_cache()

            loss = criterion(recon, outputs)
            loss_scale = criterion(std, torch.norm(recon - outputs, p = 2, dim = 1, keepdim = True).detach())
            (loss+loss_scale).backward()

            optimizer.step()
            torch.cuda.empty_cache()

            avg_loss += loss * inputs.size(0)
            avg_loss_scale += loss_scale * inputs.size(0)
            total += inputs.size(0)
            print(("\r[Epoch%d/%d]-[Batch%d/%d]-[Loss:%f]-[Loss_scale:%f]" %
                                                            (epoch+1, args.epoch_num,
                                                            i, len(train_j_dataloader),
                                                            avg_loss / total,
                                                            avg_loss_scale / total)))


        if best_loss > avg_loss and best_loss > loss:
            best_loss = avg_loss
            state_dict = {
                        'start_epoch':epoch,
                        #'optimizer':optimizer.state_dict(),
                        'transformer':transformer.state_dict(),
                        'args':args,
                        'best_loss':best_loss
                }
            torch.save(state_dict, SAVE_DIR)
            
        # EVALUATE ON TRAINING SET
        print("start evaluation on test set!")
        transformer.eval()
        score_map = []
        gt_list = []
        gt_mask_list = []
        for i,(name ,batch, ground_truth, gt) in enumerate(test_dataloader):
            with torch.no_grad():
                inputs = batch.to(device)
                ground_truth = ground_truth.to(device)
                outputs = []
                _ = backbone(inputs)
                outputs = embedding_concat(embedding_concat(outputs[0],outputs[1]),outputs[2])
                recon, std = transformer(outputs)
                
                
                
                batch_size, channels, width, height = recon.size()
                dist = torch.norm(recon - outputs, p = 2, dim = 1, keepdim = True).div(std.abs())
                dist = dist.view(batch_size, 1, width, height)
                patch_normed_score = []
                for j in range(4):
                    patch_size = pow(4, j)
                    patch_score = F.conv2d(input=dist, 
                        weight=(torch.ones(1,1,patch_size,patch_size) / (patch_size*patch_size)).to(device), 
                        bias=None, stride=patch_size, padding=0, dilation=1)
                    patch_score = F.avg_pool2d(dist,patch_size,patch_size)
                    patch_score = F.interpolate(patch_score, (width,height), mode='bilinear', align_corners=False)
                    patch_normed_score.append(patch_score)
                score = torch.zeros(batch_size,1,64,64).to(device)
                for j in range(4):
                    score = embedding_concat(score, patch_normed_score[j])
                
                score = F.conv2d(input=score, 
                        weight=torch.tensor([[[[0.0]],[[0.25]],[[0.25]],[[0.25]],[[0.25]]]]).to(device), 
                        bias=None, stride=1, padding=0, dilation=1)
                score = F.interpolate(score, (ground_truth.size(2),ground_truth.size(3)), mode='bilinear', align_corners=False)
                heatmap = score.repeat(1,3,1,1)
                score_map.append(score.cpu())
                gt_mask_list.append(ground_truth.cpu())
                gt_list.append(gt)
        
#         score_map = torch.cat(score_map,dim=0)
        
#         gt_mask_list = torch.cat(gt_mask_list,dim=0)
#         gt_list = torch.cat(gt_list,dim=0)

#         # Normalization
#         max_score = score_map.max()
#         min_score = score_map.min()
#         scores = (score_map - min_score) / (max_score - min_score)
        
#         # calculate image-level ROC AUC score
#         img_scores = scores.view(scores.size(0),-1).max(dim=1)[0]
#         gt_list = gt_list.numpy()
#         fpr, tpr, _ = roc_curve(gt_list, img_scores)
#         img_roc_auc = roc_auc_score(gt_list, img_scores)
#         print('image ROCAUC: %.3f' % (img_roc_auc))

#         # calculate per-pixel level ROCAUC
#         gt_mask = gt_mask_list.numpy().astype('int')
#         scores = scores.numpy().astype('float32')
#         fpr, tpr, thresholds = roc_curve(gt_mask.flatten(), scores.flatten()) 
#         per_pixel_rocauc = roc_auc_score(gt_mask.flatten(), scores.flatten()) 
#         print('pixel ROCAUC: %.3f' % (per_pixel_rocauc))

#         with open(os.path.join(EXPERIMENT_PATH,'args.log') ,"a") as train_log:
#             train_log.write("\r[Epoch%d]-[Loss:%f]-[Loss_scale:%f]-[image_AUC:%f]-[pixel_AUC:%f]" %
#                                                         (epoch+1, avg_loss / total, avg_loss_scale / total, img_roc_auc, per_pixel_rocauc))



    
    
    
    
    
    
    
    
    
    
    
    
#     #   Recon_score for train_ind_ls[i] 
    
    
    
    
    
    
    
    
    
    
    
    
    
    
#     recon_error_ls.append(test_score)














# # Look at recon errors of each run
# scores = np.vstack(recon_error_ls)

# #scale reconstruction errors for each run to make them comparable
# for i in range(train_sets):
#     scores[i, :] = (scores[i, :] - scores[i, train_ind_ls[i]].mean()) / scores[i, train_ind_ls[i]].std()







242
train_sets:  5
window_size:  120
[Epoch1/1]-[Batch0/60]-[Loss:0.343237]-[Loss_scale:155.271790]
[Epoch1/1]-[Batch1/60]-[Loss:0.296613]-[Loss_scale:134.087143]
[Epoch1/1]-[Batch2/60]-[Loss:0.259177]-[Loss_scale:117.110855]
[Epoch1/1]-[Batch3/60]-[Loss:0.229817]-[Loss_scale:103.822578]
[Epoch1/1]-[Batch4/60]-[Loss:0.206021]-[Loss_scale:93.070236]
[Epoch1/1]-[Batch5/60]-[Loss:0.187784]-[Loss_scale:84.838913]
[Epoch1/1]-[Batch6/60]-[Loss:0.173962]-[Loss_scale:78.603844]
[Epoch1/1]-[Batch7/60]-[Loss:0.163030]-[Loss_scale:73.671822]
[Epoch1/1]-[Batch8/60]-[Loss:0.154652]-[Loss_scale:69.890991]
[Epoch1/1]-[Batch9/60]-[Loss:0.147676]-[Loss_scale:66.741508]
[Epoch1/1]-[Batch10/60]-[Loss:0.141923]-[Loss_scale:64.142860]
[Epoch1/1]-[Batch11/60]-[Loss:0.137115]-[Loss_scale:61.969727]
[Epoch1/1]-[Batch12/60]-[Loss:0.132997]-[Loss_scale:60.106766]
[Epoch1/1]-[Batch13/60]-[Loss:0.129406]-[Loss_scale:58.480824]
[Epoch1/1]-[Batch14/60]-[Loss:0.126265]-[Loss_scale:57.057125]
[Epoch1/1]-[Batch15/60]-

In [ ]:

#save scores
np.savetxt(os.path.join(config['exp_path'], config['mode'] + '_scores.csv'), scores, delimiter=',')

# Create boolean matrix for train and test points
scores_bool = np.zeros_like(scores, dtype=bool)

for i in range(train_sets):
    scores_bool[i, train_ind_ls[i]] = True

np.savetxt(os.path.join(config['exp_path'], config['mode'] + '_scores_bool.csv'), scores_bool, delimiter=',')

#Splitt scores in train and test Scores
scores_test = scores.copy()
scores_train = scores.copy()

scores_test[scores_bool] = np.nan
scores_train[~scores_bool] = np.nan

# Check how much point is varying in train and test
scores_test_mean = np.nanmean(scores_test,axis = 0)
scores_test_std = np.nanstd(scores_test,axis = 0)
scores_train_mean = np.nanmean(scores_train,axis = 0)
scores_train_std = np.nanstd(scores_train,axis = 0)

indicator = np.abs(scores_test_mean - scores_train_mean)

# Calculate cutoff for refinement
cutoff = np.quantile(indicator, 1-assumed_contamination_rate)

data_annot = copy.deepcopy(data)
data_annot.labels = np.zeros_like(data.labels)
data_annot.labels[indicator > cutoff] = 1

# Train the model on the refined data
test_score, holdout_score = SemiSup(data_annot, holdout_data, ADtrainer, ADmodel, net, loss, logger, config)

In [ ]:
def USDR(data, holdout_data, ADtrainer, ADmodel, net, loss, logger, config):
    """
    Unsupervised Data Refinement (USDR) framework.

    Args:
        data: The training data.
        holdout_data: The holdout data for evaluation.
        ADtrainer: The anomaly detection trainer class.
        ADmodel: The anomaly detection model class.
        net: The neural network architecture.
        loss: The loss function.
        logger: The logger for logging.
        config: The configuration parameters.

    Returns:
        indicator: The indicator values.
        test_score: The test score.
        holdout_score: The holdout score.
    """
    # Extracting configuration parameters
    window_size = config['window_size']
    stride = config['stride']
    assumed_contamination_rate = config['assumed_contamination_rate']
    
    # Shuffle the data indices
    N = data.__len__()
    idx = np.arange(N)
    np.random.shuffle(idx)

    # Create train sets
    shifts = np.floor(N / stride)
    train_sets = int(shifts)
    print('train_sets: ', train_sets)
    print('window_size: ', window_size)
    train_ind_ls = []

    test_data = copy.deepcopy(data)
    test_loader = DataLoader(test_data, batch_size=config['batch_size'], shuffle=False)

    for i in range(train_sets):
        train_ind_ls.append(idx[np.arange(i * stride, i * stride + window_size) % N])

    # Train the model on each train set

    recon_error_ls = []
    
    for i in range(train_sets):
        
        train_m_dataloader = DataLoader(ImageDataset_mvtec(args,DATA_PATH,mode='train',train_paths = normal_images,test_paths = None), batch_size=2,shuffle=True,num_workers=8,drop_last=False)

        recon_error_ls.append(test_score)


    # Look at recon errors of each run
    scores = np.vstack(recon_error_ls)

    #scale reconstruction errors for each run to make them comparable
    for i in range(train_sets):
        scores[i, :] = (scores[i, :] - scores[i, train_ind_ls[i]].mean()) / scores[i, train_ind_ls[i]].std()

    #save scores
    np.savetxt(os.path.join(config['exp_path'], config['mode'] + '_scores.csv'), scores, delimiter=',')

    # Create boolean matrix for train and test points
    scores_bool = np.zeros_like(scores, dtype=bool)

    for i in range(train_sets):
        scores_bool[i, train_ind_ls[i]] = True

    np.savetxt(os.path.join(config['exp_path'], config['mode'] + '_scores_bool.csv'), scores_bool, delimiter=',')

    #Splitt scores in train and test Scores
    scores_test = scores.copy()
    scores_train = scores.copy()

    scores_test[scores_bool] = np.nan
    scores_train[~scores_bool] = np.nan

    # Check how much point is varying in train and test
    scores_test_mean = np.nanmean(scores_test,axis = 0)
    scores_test_std = np.nanstd(scores_test,axis = 0)
    scores_train_mean = np.nanmean(scores_train,axis = 0)
    scores_train_std = np.nanstd(scores_train,axis = 0)

    indicator = np.abs(scores_test_mean - scores_train_mean)
    
    # Calculate cutoff for refinement
    cutoff = np.quantile(indicator, 1-assumed_contamination_rate)

    data_annot = copy.deepcopy(data)
    data_annot.labels = np.zeros_like(data.labels)
    data_annot.labels[indicator > cutoff] = 1

    # Train the model on the refined data
    test_score, holdout_score = SemiSup(data_annot, holdout_data, ADtrainer, ADmodel, net, loss, logger, config)

    return indicator, test_score, holdout_score